In [141]:
import tensorflow as tf
print("Usandor Tensorflow version " + tf.__version__)


if tf.test.gpu_device_name():
  print('Usando GPU: {}'.format(tf.test.gpu_device_name()))
else:
  print("Usando CPU.")

Usandor Tensorflow version 2.13.1
Usando CPU.


In [142]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [143]:
NUMBER_OF_LAYERS = [1]
NUMBER_OF_NEURONS = [[50,50,50,50]]
NUMBER_OF_STEPS = [1,4,6,10]

In [144]:
from numpy import array, hstack
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import pandas as pd
from empresa4.datasets import get_dataset, nombres_datasets
from keras.callbacks import EarlyStopping
from datetime import datetime
from empresa4.core import calculate_error

In [145]:
# Function to split a single time series into overlapping sequences
def split_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        end_ix = i + n_steps
        if end_ix > len(sequences):
            break
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix - 1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)


In [146]:
# Read the data
# df = get_dataset("02_productos_todos_anti_leak")
df = pd.read_csv("./fake.csv")
df

{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()


,periodo,product_id,cust_request_qty,cust_request_tn,product_category,cat2,sku_size,plan_precios_cuidados,tn
0,201701,20001,1.0,2.0,FOODS,ADEREZOS,3.0,1,6.0
1,201702,20001,2.0,3.0,FOODS,ADEREZOS,4.0,1,8.0
2,201703,20001,3.0,4.0,FOODS,ADEREZOS,5.0,1,10.0
3,201704,20001,4.0,5.0,FOODS,ADEREZOS,6.0,1,12.0
4,201705,20001,5.0,6.0,FOODS,ADEREZOS,7.0,1,14.0
...,...,...,...,...,...,...,...,...,...
175,201908,20005,98.0,103.0,HC,ROPA LAVADO,108.0,1,1080.0
176,201909,20005,101.0,106.0,HC,ROPA LAVADO,111.0,1,1110.0
177,201910,20005,104.0,109.0,HC,ROPA LAVADO,114.0,1,1140.0
178,201911,20005,107.0,112.0,HC,ROPA LAVADO,117.0,1,1170.0


In [147]:
df[df["product_id"] == 20001]

,periodo,product_id,cust_request_qty,cust_request_tn,product_category,cat2,sku_size,plan_precios_cuidados,tn
0,201701,20001,1.0,2.0,FOODS,ADEREZOS,3.0,1,6.0
1,201702,20001,2.0,3.0,FOODS,ADEREZOS,4.0,1,8.0
2,201703,20001,3.0,4.0,FOODS,ADEREZOS,5.0,1,10.0
3,201704,20001,4.0,5.0,FOODS,ADEREZOS,6.0,1,12.0
4,201705,20001,5.0,6.0,FOODS,ADEREZOS,7.0,1,14.0
5,201706,20001,6.0,7.0,FOODS,ADEREZOS,8.0,1,16.0
6,201707,20001,7.0,8.0,FOODS,ADEREZOS,9.0,1,18.0
7,201708,20001,8.0,9.0,FOODS,ADEREZOS,10.0,1,20.0
8,201709,20001,9.0,10.0,FOODS,ADEREZOS,11.0,1,22.0
9,201710,20001,10.0,11.0,FOODS,ADEREZOS,12.0,1,24.0


In [148]:
equiv = {
    201701: 201703,
    201702: 201704,
    201703: 201705,
    201704: 201706,
    201705: 201707,
    201706: 201708,
    201707: 201709,
    201708: 201710,
    201709: 201711,
    201710: 201712,
    201711: 201801,
    201712: 201802,
    201801: 201803,
    201802: 201804,
    201803: 201805,
    201804: 201806,
    201805: 201807,
    201806: 201808,
    201807: 201809,
    201808: 201810,
    201809: 201811,
    201810: 201812,
    201811: 201901,
    201812: 201902,
    201901: 201903,
    201902: 201904,
    201903: 201905,
    201904: 201906,
    201905: 201907,
    201906: 201908,
    201907: 201909,
    201908: 201910,
    201909: 201911,
    201910: 201912,
    201911: 202001,
    201912: 202002,
    202001: 202003,
    202002: 202004,
}
target_df = get_dataset("02_productos_todos")


def lag_target_class(row):
    # from the column "periodo" and "product_id" of this row, get the equivalen periodo in equiv and get the tn column from "target" df for this product_id and the equiv periodo
    product_id = row["product_id"]
    periodo = row["periodo"]
    periodo_equiv = equiv.get(periodo)
    if periodo_equiv is None:
        return None
    value = target_df[(target_df["product_id"] == product_id) & (target_df["periodo"] == periodo_equiv)]["tn"].values[0]
    return value


def lag_target_class_2(row):
    # from the column "periodo" and "product_id" of this row, get the equivalen periodo in equiv and get the tn column from "target" df for this product_id and the equiv periodo
    product_id = row["product_id"]
    periodo = row["periodo"]
    periodo_equiv = equiv.get(periodo)
    if periodo_equiv is None:
        return None
    if periodo not in [201911, 201912]:
        value = df[(df["product_id"] == product_id) & (df["periodo"] == periodo_equiv)]["tn"].values[0]
    else:
        value = None
    return value

{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()


In [149]:
df["target"] = df.apply(lag_target_class_2, axis=1)
df

,periodo,product_id,cust_request_qty,cust_request_tn,product_category,cat2,sku_size,plan_precios_cuidados,tn,target
0,201701,20001,1.0,2.0,FOODS,ADEREZOS,3.0,1,6.0,10.0
1,201702,20001,2.0,3.0,FOODS,ADEREZOS,4.0,1,8.0,12.0
2,201703,20001,3.0,4.0,FOODS,ADEREZOS,5.0,1,10.0,14.0
3,201704,20001,4.0,5.0,FOODS,ADEREZOS,6.0,1,12.0,16.0
4,201705,20001,5.0,6.0,FOODS,ADEREZOS,7.0,1,14.0,18.0
...,...,...,...,...,...,...,...,...,...,...
175,201908,20005,98.0,103.0,HC,ROPA LAVADO,108.0,1,1080.0,1140.0
176,201909,20005,101.0,106.0,HC,ROPA LAVADO,111.0,1,1110.0,1170.0
177,201910,20005,104.0,109.0,HC,ROPA LAVADO,114.0,1,1140.0,1200.0
178,201911,20005,107.0,112.0,HC,ROPA LAVADO,117.0,1,1170.0,NaN


In [150]:
df[df["product_id"] == 20001]

,periodo,product_id,cust_request_qty,cust_request_tn,product_category,cat2,sku_size,plan_precios_cuidados,tn,target
0,201701,20001,1.0,2.0,FOODS,ADEREZOS,3.0,1,6.0,10.0
1,201702,20001,2.0,3.0,FOODS,ADEREZOS,4.0,1,8.0,12.0
2,201703,20001,3.0,4.0,FOODS,ADEREZOS,5.0,1,10.0,14.0
3,201704,20001,4.0,5.0,FOODS,ADEREZOS,6.0,1,12.0,16.0
4,201705,20001,5.0,6.0,FOODS,ADEREZOS,7.0,1,14.0,18.0
5,201706,20001,6.0,7.0,FOODS,ADEREZOS,8.0,1,16.0,20.0
6,201707,20001,7.0,8.0,FOODS,ADEREZOS,9.0,1,18.0,22.0
7,201708,20001,8.0,9.0,FOODS,ADEREZOS,10.0,1,20.0,24.0
8,201709,20001,9.0,10.0,FOODS,ADEREZOS,11.0,1,22.0,26.0
9,201710,20001,10.0,11.0,FOODS,ADEREZOS,12.0,1,24.0,28.0


In [151]:
df["product_id"].nunique()

5

In [152]:
df["periodo"].value_counts()

{}
()


periodo
201701    5
201702    5
201809    5
201810    5
201811    5
201812    5
201901    5
201902    5
201903    5
201904    5
201905    5
201906    5
201907    5
201908    5
201909    5
201910    5
201911    5
201808    5
201807    5
201806    5
201709    5
201703    5
201704    5
201705    5
201706    5
201707    5
201708    5
201710    5
201805    5
201711    5
201712    5
201801    5
201802    5
201803    5
201804    5
201912    5
Name: count, dtype: int64

In [153]:
# Sort by `periodo` just to be sure
product_data = df.sort_values(["product_id", "periodo"])

{'axis': -1, 'kind': None, 'order': None}
()


In [154]:
product_data.shape, product_data.columns

((180, 10),
 Index(['periodo', 'product_id', 'cust_request_qty', 'cust_request_tn',
        'product_category', 'cat2', 'sku_size', 'plan_precios_cuidados', 'tn',
        'target'],
       dtype='object'))

In [155]:
product_data.fillna(0, inplace=True)

In [156]:
# Using scikit-learn, ONE HOT ENCODE the categorical variables: product_category
product_data = pd.get_dummies(product_data, columns=["product_category", "cat2"])

# sort columns so that 'lag_tn' is the last column in the dataframe
product_data = product_data[[col for col in product_data.columns if col != 'target'] + ['target']]

In [157]:
# convert boolean columns to int
product_data = product_data.astype({"product_category_FOODS": int, "product_category_HC": int, "product_category_PC": int})

In [158]:
# convert bool columns to int
values = [
    "ROPA LAVADO",
    "PIEL1",
    "ADEREZOS",
]

for value in values:
    product_data = product_data.astype({f"cat2_{value}": int})
product_data.dtypes.sort_values()

{}
()


periodo                     int64
product_id                  int64
plan_precios_cuidados       int64
product_category_FOODS      int64
product_category_HC         int64
product_category_PC         int64
cat2_ADEREZOS               int64
cat2_PIEL1                  int64
cat2_ROPA LAVADO            int64
cust_request_qty          float64
cust_request_tn           float64
sku_size                  float64
tn                        float64
target                    float64
dtype: object

In [159]:
product_data.isna().sum().sum()

0

In [160]:
# Split the data into training and test sets
train_df = product_data[product_data["periodo"] <= 201909]
test_df = product_data[product_data["periodo"] <= 201910]
predict_df = product_data[product_data["periodo"] <= 201912]

{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()


In [161]:
# standarize the data
from sklearn.preprocessing import StandardScaler

# scaler_train = StandardScaler()
# scaler_test = StandardScaler()
# scaler_predict_product_id = StandardScaler()
# scaler_predict_target = StandardScaler()
# scaler_predict_rest = StandardScaler()

rest = [col for col in predict_df.columns if col not in ["product_id", "target"]]

# train_df[train_df.columns] = scaler_train.fit_transform(train_df[train_df.columns])
# test_df[test_df.columns] = scaler_test.fit_transform(test_df[test_df.columns])
# predict_df[rest] = scaler_predict_rest.fit_transform(predict_df[rest])
# predict_df["target"] = scaler_predict_target.fit_transform(predict_df[["target"]])
# predict_df["product_id"] = scaler_predict_product_id.fit_transform(predict_df[["product_id"]])

In [162]:
def train_model(n_layers, n_neurons, dropout_threshold, n_steps, X_train, y_train, X_test, y_test):
    # Train a single model where product_id is part of the input
    print(f"Training model with {n_layers} layers, {n_neurons} neurons, {n_steps} steps.")

    # Number of features 
    n_features = X_train.shape[2]

    # Define the LSTM model
    if n_layers == 1:
        model = Sequential()
        model.add(LSTM(n_neurons, activation="relu", input_shape=(n_steps, n_features)))
        model.add(Dense(1))
        model.compile(optimizer="adam", loss="mse")
    elif n_layers == 2:
        model = Sequential()
        model.add(LSTM(n_neurons, activation="relu", return_sequences=True, input_shape=(n_steps, n_features)))
        model.add(Dropout(dropout_threshold))
        model.add(LSTM(n_neurons, activation="relu"))
        model.add(Dense(1))
        model.compile(optimizer="adam", loss="mse")
    elif n_layers == 3:
        model = Sequential()
        model.add(LSTM(n_neurons, activation="relu", return_sequences=True, input_shape=(n_steps, n_features)))
        model.add(Dropout(dropout_threshold))
        model.add(LSTM(n_neurons, activation="relu", return_sequences=True))
        model.add(Dropout(dropout_threshold))
        model.add(LSTM(n_neurons, activation="relu"))
        model.add(Dense(1))
        model.compile(optimizer="adam", loss="mse")
    elif n_layers == 4:
        model = Sequential()
        model.add(LSTM(n_neurons, activation="relu", return_sequences=True, input_shape=(n_steps, n_features)))
        model.add(Dropout(dropout_threshold))
        model.add(LSTM(n_neurons, activation="relu", return_sequences=True))
        model.add(Dropout(dropout_threshold))
        model.add(LSTM(n_neurons, activation="relu", return_sequences=True))
        model.add(Dropout(dropout_threshold))
        model.add(LSTM(n_neurons, activation="relu"))
        model.add(Dense(1))
        model.compile(optimizer="adam", loss="mse")
    else:
        raise ValueError("n_layers must be 1, 2, 3 or 4")

    # Define EarlyStopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Fit the model
    model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), callbacks=[early_stopping], verbose=1)
    return model

In [163]:
def make_predictions(n_steps, model, X_predict, y_predict):
    rows = []
    # for each product_id, generate a prediction for 201904
    for i, product in enumerate(predict_df["product_id"].unique()):
        print(
            f"Predicting 201904 for product {product} ({i+1}/{len(predict_df['product_id'].unique())}))"
        )

        this_predict_df = predict_df[predict_df["product_id"] == product]

        this_predict_df_array = this_predict_df.values

        n_features = X_predict.shape[2]

        # Prepare the input for prediction
        x_input = this_predict_df_array[-n_steps:, :-1]
        x_input = x_input.reshape((1, n_steps, n_features))

        print("/////////// INPUT ///////////////////////////")
        print("/////////// INPUT ///////////////////////////")
        print("/////////// INPUT ///////////////////////////")
        print(x_input)
        print("/////////// INPUT ///////////////////////////")
        print("/////////// INPUT ///////////////////////////")
        print("/////////// INPUT ///////////////////////////")


        # Make prediction
        yhat = model.predict(x_input, verbose=0)
        if yhat[0][0] < 1:
            yhat[0][0] = 0

        # Append to final output DataFrame
        rows.append(
            {
                "product_id": product,
                "target": yhat[0][0],
            }
        )
        
    final_output = pd.DataFrame(
        rows,
        columns=["product_id", "target"],
    )
    final_output = final_output.sort_values("product_id", ascending=True)
    timestamp = datetime.now().timestamp()
    final_output.to_csv(f"./output/output_lstm8_BO_{timestamp}.csv", index=False)
    print("//////////////////////////////////////")
    print(X_predict.shape, y_predict.shape, final_output.shape)
    print("//////////////////////////////////////")
    return final_output

In [164]:
def calculate_score(predictions):
    # revert the standarization for the output
    # predictions["target"] = scaler_predict_target.inverse_transform(predictions[["target"]])
    # predictions["product_id"] = scaler_predict_product_id.inverse_transform(predictions[["product_id"]])
    
    predictions["product_id"] = predictions["product_id"].astype(int)
    predictions.rename(columns={"target": "prediction"}, inplace=True)

    print("*********"*10)
    print("*********"*10)
    print("*********"*10)
    print(predictions)
    print("*********"*10)
    print("*********"*10)
    print("*********"*10)

    
    return calculate_error(predictions["prediction"].tolist(), [76, 118.5, 75, 190, 1200])


In [165]:
# g_s_rows = []
# for n_layers in NUMBER_OF_LAYERS:
#     for n_neurons in NUMBER_OF_NEURONS:
#         for n_steps in NUMBER_OF_STEPS:

#             X_train, y_train = split_sequences(train_df.values, n_steps)
#             X_test, y_test = split_sequences(test_df.values, n_steps)
#             X_predict, y_predict = split_sequences(predict_df.values, n_steps)

#             model = train_model(n_layers, n_neurons, n_steps, X_train, y_train, X_test, y_test)
#             predictions = make_predictions(n_steps, model, X_predict, y_predict)
#             score = calculate_score(predictions)
#             print("SCORE: ", score)

#             g_s_rows.append(
#                 {
#                     "n_layers": n_layers,
#                     "n_neurons": n_neurons,
#                     "n_steps": n_steps,
#                     "score": score,
#                 }
#             )

#             g_s_df = pd.DataFrame(g_s_rows)
#             g_s_df.to_csv(f"./output/test_grid_search_lstm8_productos_{n_layers}_{n_neurons[0]}_{n_steps}_{score}.csv", index=False)

# Bayesian Optimization

In [166]:
import pandas as pd
import optuna

In [167]:
def objective(trial):
    # Define the search space using Optuna
    n_layers = trial.suggest_int('n_layers', 1, 4)
    n_neurons = trial.suggest_int('n_neurons', 20, 500)
    n_steps = trial.suggest_int('n_steps', 1, 20)
    embudo = trial.suggest_float('embudo', 1.1, 2.5)
    dropout_threshold = trial.suggest_float('dropout_threshold', 0.1, 0.66)

    # Your existing logic to train and evaluate the model
    X_train, y_train = split_sequences(train_df.values, n_steps)
    X_test, y_test = split_sequences(test_df.values, n_steps)
    X_predict, y_predict = split_sequences(predict_df.values, n_steps)

    model = train_model(n_layers, n_neurons, dropout_threshold, n_steps, X_train, y_train, X_test, y_test)
    predictions = make_predictions(n_steps, model, X_predict, y_predict)
    score = calculate_score(predictions)

    # Save the results
    result = {
        "n_layers": n_layers,
        "n_neurons": n_neurons,
        "n_steps": n_steps,
        "dropout_threshold": dropout_threshold,
        "score": score
    }
    g_s_df = pd.DataFrame([result])
    g_s_df.to_csv(f"./output/lstm_8_optuna_{n_layers}_{n_neurons}_{n_steps}_{dropout_threshold}_{score}.csv", mode='a', index=False)

    return score

In [168]:
# Create a study object and specify the optimization direction
study = optuna.create_study(direction='minimize', study_name="lstm_8_BO_test")

# Perform optimization
study.optimize(objective, n_trials=50)

# Extract the best trial
best_trial = study.best_trial

print('Best trial:')
for key, value in best_trial.params.items():
    print(f"{key}: {value}")

[I 2023-11-24 19:46:26,159] A new study created in memory with name: lstm_8_BO_test


{'out': None}
(1, 4)
{'out': None}
(20, 500)
{'out': None}
(1, 20)
Training model with 1 layers, 498 neurons, 18 steps.
Epoch 1/200


5/5 [==============================] - 1s 47ms/step - loss: 764821248.0000 - val_loss: 3197652736.0000
Epoch 2/200
5/5 [==============================] - 0s 23ms/step - loss: 1306525312.0000 - val_loss: 196674448.0000
Epoch 3/200
5/5 [==============================] - 0s 21ms/step - loss: 793546112.0000 - val_loss: 422885984.0000
Epoch 4/200
5/5 [==============================] - 0s 21ms/step - loss: 822433024.0000 - val_loss: 217925296.0000
Epoch 5/200
5/5 [==============================] - 0s 22ms/step - loss: 191734160.0000 - val_loss: 289749472.0000
Epoch 6/200
5/5 [==============================] - 0s 21ms/step - loss: 530551008.0000 - val_loss: 434444096.0000
Epoch 7/200
5/5 [==============================] - 0s 22ms/step - loss: 750432896.0000 - val_loss: 2191541504.0000
Epoch 8/200
5/5 [==============================] - 0s 21ms/step - loss: 690891648.0000 - val_loss: 68135264.0000
Epoch 9/200
5/5 [==============================] - 0s 21ms/step - loss: 131686536.0000 - val_loss:

[I 2023-11-24 19:46:29,421] Trial 0 finished with value: 5.000160504623004 and parameters: {'n_layers': 1, 'n_neurons': 498, 'n_steps': 18, 'dropout_threshold': 0.6147287077041598}. Best is trial 0 with value: 5.000160504623004.


/////////// INPUT ///////////////////////////
/////////// INPUT ///////////////////////////
/////////// INPUT ///////////////////////////
[[[2.01807e+05 2.00030e+04 1.85000e+01 1.95000e+01 2.05000e+01
   1.00000e+00 4.10000e+01 0.00000e+00 0.00000e+00 1.00000e+00
   0.00000e+00 1.00000e+00 0.00000e+00]
  [2.01808e+05 2.00030e+04 1.95000e+01 2.05000e+01 2.15000e+01
   1.00000e+00 4.30000e+01 0.00000e+00 0.00000e+00 1.00000e+00
   0.00000e+00 1.00000e+00 0.00000e+00]
  [2.01809e+05 2.00030e+04 2.05000e+01 2.15000e+01 2.25000e+01
   1.00000e+00 4.50000e+01 0.00000e+00 0.00000e+00 1.00000e+00
   0.00000e+00 1.00000e+00 0.00000e+00]
  [2.01810e+05 2.00030e+04 2.15000e+01 2.25000e+01 2.35000e+01
   1.00000e+00 4.70000e+01 0.00000e+00 0.00000e+00 1.00000e+00
   0.00000e+00 1.00000e+00 0.00000e+00]
  [2.01811e+05 2.00030e+04 2.25000e+01 2.35000e+01 2.45000e+01
   1.00000e+00 4.90000e+01 0.00000e+00 0.00000e+00 1.00000e+00
   0.00000e+00 1.00000e+00 0.00000e+00]
  [2.01812e+05 2.00030e+04 2.350

...

Predicting 201904 for product 20005 (5/5))
/////////// INPUT ///////////////////////////
/////////// INPUT ///////////////////////////
/////////// INPUT ///////////////////////////
[[[2.01911e+05 2.00050e+04 1.07000e+02 1.12000e+02 1.17000e+02
   1.00000e+00 1.17000e+03 0.00000e+00 1.00000e+00 0.00000e+00
   0.00000e+00 0.00000e+00 1.00000e+00]
  [2.01912e+05 2.00050e+04 1.10000e+02 1.15000e+02 1.20000e+02
   1.00000e+00 1.20000e+03 0.00000e+00 1.00000e+00 0.00000e+00
   0.00000e+00 0.00000e+00 1.00000e+00]]]
/////////// INPUT ///////////////////////////
/////////// INPUT ///////////////////////////
/////////// INPUT ///////////////////////////
{}
()
//////////////////////////////////////
(179, 2, 13) (179,) (5, 2)
//////////////////////////////////////
******************************************************************************************
******************************************************************************************
******************************************************

[I 2023-11-24 19:51:12,630] Trial 48 finished with value: 0.1746122584927689 and parameters: {'n_layers': 1, 'n_neurons': 20, 'n_steps': 5, 'dropout_threshold': 0.2538654962810635}. Best is trial 46 with value: 0.05197422617205154.


{}
()
//////////////////////////////////////
(176, 5, 13) (176,) (5, 2)
//////////////////////////////////////
******************************************************************************************
******************************************************************************************
******************************************************************************************
   product_id   prediction
0       20001   126.805107
1       20002   165.991486
2       20003   125.351448
3       20004   284.376862
4       20005  1153.255859
******************************************************************************************
******************************************************************************************
******************************************************************************************
{'axis': -1, 'kind': None, 'order': None}
()
{'axis': -1, 'kind': None, 'order': None}
()
{'out': None}
(0.5714285714285714, 4.0)
{'axis': -1, 'kind': None, 'order': None}
()
{'axis

[I 2023-11-24 19:51:18,545] Trial 49 finished with value: 1.0 and parameters: {'n_layers': 2, 'n_neurons': 446, 'n_steps': 18, 'dropout_threshold': 0.1797248303070684}. Best is trial 46 with value: 0.05197422617205154.


Predicting 201904 for product 20005 (5/5))
/////////// INPUT ///////////////////////////
/////////// INPUT ///////////////////////////
/////////// INPUT ///////////////////////////
[[[2.01807e+05 2.00050e+04 5.90000e+01 6.40000e+01 6.90000e+01
   1.00000e+00 6.90000e+02 0.00000e+00 1.00000e+00 0.00000e+00
   0.00000e+00 0.00000e+00 1.00000e+00]
  [2.01808e+05 2.00050e+04 6.20000e+01 6.70000e+01 7.20000e+01
   1.00000e+00 7.20000e+02 0.00000e+00 1.00000e+00 0.00000e+00
   0.00000e+00 0.00000e+00 1.00000e+00]
  [2.01809e+05 2.00050e+04 6.50000e+01 7.00000e+01 7.50000e+01
   1.00000e+00 7.50000e+02 0.00000e+00 1.00000e+00 0.00000e+00
   0.00000e+00 0.00000e+00 1.00000e+00]
  [2.01810e+05 2.00050e+04 6.80000e+01 7.30000e+01 7.80000e+01
   1.00000e+00 7.80000e+02 0.00000e+00 1.00000e+00 0.00000e+00
   0.00000e+00 0.00000e+00 1.00000e+00]
  [2.01811e+05 2.00050e+04 7.10000e+01 7.60000e+01 8.10000e+01
   1.00000e+00 8.10000e+02 0.00000e+00 1.00000e+00 0.00000e+00
   0.00000e+00 0.00000e+00 1.